<a href="https://colab.research.google.com/github/TawnyVTC/EDA/blob/main/models_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelos Benchmarks**

In [1]:
# Librerias Importadas

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.stattools import jarque_bera



Se importa la base de datos y definimos las variables predictoras y la variable respuesta. Luego, dividimos los datos para formar el conjunto de entrenamiento y el de prueba.

In [ ]:
# Cargar datos
df = pd.read_csv(r"C:\Users\TAWTOCA\OneDrive - Universidad del Norte\Documentos\Visual Folders\Visual Files\Rentabilidad_filtrado.csv")

# Definir variables predictoras y respuesta
X = df[["APORTES_RECIBIDOS", "PRECIERRE_FONDO_DIA_T", "TIPO_PARTICIPACION", "NUMERO_INVERSIONISTAS", "TIPO_ENTIDAD"]]
y = df["RETIROS_REDENCIONES"]



# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
modelos = {
    'lineal': LinearRegression(),
    'ridge': Ridge(alpha=1.0),
    'lasso': Lasso(alpha=0.1),
    'knn': KNeighborsRegressor(n_neighbors=5),
    'random_forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'xgboost': XGBRegressor(n_estimators=100, random_state=42)
}


In [ ]:
def fit(nombre_modelo, modelos, X_train, y_train, X_test, y_test):
    """
    Entrena y evalúa un modelo especificado por su nombre simplificado.

    Parámetros:
    - nombre_modelo: str, clave del modelo en el diccionario.
    - modelos: diccionario con nombres simplificados como claves y modelos sklearn como valores.
    - X_train, y_train: datos de entrenamiento.
    - X_test, y_test: datos de prueba.

    Retorna:
    - resultados: diccionario con tiempo, train_score y test_score.
    """
    if nombre_modelo not in modelos:
        raise ValueError(f"Modelo '{nombre_modelo}' no encontrado. Modelos disponibles: {list(modelos.keys())}")

    modelo = modelos[nombre_modelo]

    st = time.time()
    modelo.fit(X_train, y_train)
    et = time.time()

    resultados = {
        'modelo': nombre_modelo,
        'tiempo_entrenamiento': round(et - st, 2),
        'train_score': round(modelo.score(X_train, y_train), 2),
        'test_score': round(modelo.score(X_test, y_test), 2)
    }

    print(f"\nModelo: {nombre_modelo}")
    print("  Tiempo de entrenamiento: {:.2f} segundos".format(resultados['tiempo_entrenamiento']))
    print("  Training set score (R²): {:.2f}".format(resultados['train_score']))
    print("  Test set score (R²): {:.2f}".format(resultados['test_score']))



## **GridSearch**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Definir los hiperparámetros como distribuciones
param_distributions = {
    'n_estimators': randint(50, 150),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# RandomizedSearch
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_distributions,
    n_iter=20,       # Solo 20 combinaciones al azar
    cv=3,            # 3 folds (más rápido)
    n_jobs=-1,       # Usa todos los núcleos, cuidado con la RAM
    scoring='neg_mean_squared_error',
    verbose=2,
    random_state=42
)

# Entrenar sobre la muestra
random_search.fit(X_train, y_train)

# Resultados
print("Mejores parámetros encontrados:")
print(random_search.best_params_)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:
results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv('random_search_results.csv', index=False)

## **Modelos**

In [ ]:
fit("lineal", modelos, X_train, y_train, X_test, y_test)



Modelo: lineal
  Tiempo de entrenamiento: 0.06 segundos
  Training set score (R²): 0.72
  Test set score (R²): 0.70


In [ ]:
fit("ridge", modelos, X_train, y_train, X_test, y_test)



Modelo: ridge
  Tiempo de entrenamiento: 0.03 segundos
  Training set score (R²): 0.72
  Test set score (R²): 0.70


In [ ]:
fit("lasso", modelos, X_train, y_train, X_test, y_test)



Modelo: lasso
  Tiempo de entrenamiento: 0.55 segundos
  Training set score (R²): 0.72
  Test set score (R²): 0.70


In [ ]:
fit("knn", modelos, X_train, y_train, X_test, y_test)



Modelo: knn
  Tiempo de entrenamiento: 0.99 segundos
  Training set score (R²): 0.78
  Test set score (R²): 0.74


In [ ]:
fit("random_forest", modelos, X_train, y_train, X_test, y_test)



Modelo: random_forest
  Tiempo de entrenamiento: 367.10 segundos
  Training set score (R²): 0.96
  Test set score (R²): 0.77


In [ ]:
fit("xgboost", modelos, X_train, y_train, X_test, y_test)


Modelo: xgboost
  Tiempo de entrenamiento: 1.37 segundos
  Training set score (R²): 0.84
  Test set score (R²): 0.69
